In [133]:
import minsearch
import os
import json
from groq import Groq

# Directory containing all JSON files
json_folder_path = 'D:\\NBEDirectivesAssist\\ConvertedToJson\\'

# List to hold all documents from all JSON files
all_documents = []

# Loop through each JSON file in the directory
for json_file in os.listdir(json_folder_path):
    if json_file.endswith(".json"):
        json_file_path = os.path.join(json_folder_path, json_file)
        with open(json_file_path, 'rt', encoding='utf-8') as f_in:
            docs_raw = json.load(f_in)
        
        # Extract documents from the current JSON file
        documents = []
        for directive_id_dict in docs_raw:
            for doc in directive_id_dict['sections']:
                # Add additional fields to the doc
                doc['document_id'] = directive_id_dict['document_id']
                doc['title'] = directive_id_dict['title']
                documents.append(doc)

        # Append documents from this file to the global list
        all_documents.extend(documents)

# Index the documents (from all files)
index = minsearch.Index(
    text_fields=["section_title", "content", "title"],
    keyword_fields=["document_id"]
)

# Index the combined documents
index.fit(all_documents)

# Example query
q = 'what is the impact of the foreign exchange directive for Ethiopian banks?'

# Boosting specific fields
boost = {'content': 10.0, 'title': 2, 'section_title': 0.5}

# Search for the query, filtering by a specific directive
result = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={'title': 'Asset Classification and Provisioning Directive'}
)

# Create the prompt for the LLM
prompt_template = """
You're National Bank of Ethiopia Directives assistant. Answer the QUESTION based on the CONTEXT from the directive document.
Use  facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, Answer from your general knowledge.

QUESTION: {question}
CONTEXT: {context}
""".strip()

# Generate the context from the search results
context = ""

for doc in result:
    context += f"section_id: {doc['section_id']}\nsection_title: {doc['section_title']}\npage_number: {doc['page_number']}\ncontent: {doc['content']}\ndocument_id: {doc['document_id']}\ntitle: {doc['title']}\n\n"

# Format the prompt
prompt = prompt_template.format(question=q, context=context).strip()

# Initialize Groq client
client = Groq(api_key="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")

# Get the answer from the LLM model
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)

# Output the result
print(chat_completion.choices[0].message.content)


Based on the provided text, it can be inferred that the impact of the foreign exchange directive for Ethiopian banks is significant. The directive aims to regulate and standardize the foreign exchange market in Ethiopia, ensuring transparency, stability, and efficiency in foreign exchange transactions.

Some of the impacts of the directive on Ethiopian banks include:

1. Regulation of foreign exchange transactions: The directive establishes regulations for foreign exchange transactions, including reporting requirements, exchange rates, and usage of foreign currencies.
2. Standardization of exchange rates: The directive introduces the concept of the indicative daily exchange rate, which serves as a reference price for foreign exchange transactions. This helps to standardize exchange rates and ensures that banks are using the same rates for their transactions.
3. Increased transparency: The directive requires banks to report their exchange rates and foreign exchange transactions, providi